In [1]:
# %% loading keras LSTM and other libs
from keras.models import Sequential
from keras.layers import Dense, LSTM, RNN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2023-04-27 12:53:59.855471: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-27 12:54:00.177985: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 12:54:01.606263: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2023-04-27 12:54:03.673782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-27 12:54:03.823007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-27 12:54:03.823370: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
# manual training data
data = pd.read_csv('datasets/graduation_rate.csv')
data1 = np.array(data['ACT composite score'])
data2 = np.array(data['SAT total score'])
data3 = np.array(data['parental income'])
data4 = np.array(data['high school gpa'])
data5 = np.array(data['college gpa'])
data6 = np.array(data['years to graduate'])
def segmentation(start,end):
    X = []
    y = []
    for i in range(start,end):
        segment = [[data1[i]],[data2[i]],[data3[i]],[data4[i]],[data5[i]]]
        X.append(segment)
        y.append([(data6[i])])
    return np.array(X),np.array(y)


In [3]:
segment_length = len(data)
divide = int(len(data)*0.8)
X_train , y_train = segmentation(0,divide)
X_test , y_test = segmentation(divide,len(data))

print(X_train.shape)
# print(len(X_train))
# print(y_train)
# print(len(y_train))
# X_train = np.array([[[0,1,0,1]],[[0,0,1,1]]])
# print(X_train.shape)
# y_train = np.array([[0],[1]])
# print(y_train.shape)

(800, 5, 1)


In [4]:
# Creating the neural network model
model = Sequential()

# Adding the layers to the model
model.add(LSTM(64, input_shape=(5,1))) # LSTM layer with 8 neurons, input shape of (1, 4)
model.add(Dense(32, activation='relu')) # Hidden layer with neurons
model.add(Dense(32, activation='relu')) # Hidden layer with neurons
model.add(Dense(32, activation='relu')) # Hidden layer with neurons
model.add(Dense(32, activation='relu')) # Hidden layer with neurons
model.add(Dense(32, activation='relu')) # Hidden layer with neurons
model.add(Dense(32, activation='relu')) # Hidden layer with neurons
model.add(Dense(1, activation='linear')) # Output layer with 1 neuron

linear_loss ='mean_squared_error'
sigmoid_loss = 'binary_crossentropy'

# Compiling the model
model.compile(loss=linear_loss, optimizer='RMSprop', metrics=['accuracy'])

2023-04-27 12:54:15.057774: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-27 12:54:15.057967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-27 12:54:15.058066: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=5)
plt.plot(history.history['loss'], label='Training Loss')
plt.legend()
plt.show()
# %%
# loading test data
# X_test = np.array([[[0,1,0,1]],[[0,0,1,1]]])
# y_test = np.array([[0],[1]])
# evaluating model
score = model.evaluate(X_test, y_test, batch_size=1)
print("Accuracy: %.2f%%" % (score[0]*100))

In [ ]:
# printing pridiction score
y_pred = model.predict(X_test)
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.show()

In [ ]:
pred_data = np.array([[[0],[0],[0],[4.0]]])
y_pred = model.predict(pred_data)
y_pred

In [ ]:
# Save the trained model as a pickle string.
saved_model = pickle.dumps(model)
# %%
saved_model
# %%
file = open(input("enter name of model:"),"wb")
file.write(saved_model)
# %%
